In [ ]:
%pylab inline
import networkx as nx

from sklearn.datasets import make_sparse_spd_matrix

from regain import datasets, plot; reload(datasets)
from regain.datasets import is_pos_semidef, is_pos_def

In [ ]:
import matplotlib.pylab as plt
x = np.linspace(-np.pi, np.pi, 10)
y = np.sin((x))*2
print(y)
plt.plot(x,y )
plt.xlabel('Angle [rad]')
plt.ylabel('sin(x)')
plt.axis('tight')
plt.ylim((0,1))
plt.show()

In [ ]:
from regain.datasets import make_ell

In [ ]:
def generate_dataset_sin_cos(n_dim_obs=100, n_dim_lat=10, T=10, sparsity=0.95):
    L, K_OH = make_ell(n_dim_obs, n_dim_lat=n_dim_lat)
    
    phase = np.random.randn(n_dim_obs, n_dim_obs)*np.pi
    phase[np.triu_indices(n_dim_obs)[::-1]] = phase[np.triu_indices(n_dim_obs)]

    clip = np.zeros((n_dim_obs, n_dim_obs))
    picks = np.random.permutation(len(np.triu_indices(n_dim_obs,1)[0]))
    dim = int(len(np.triu_indices(n_dim_obs,1)[0])*sparsity)
    picks = picks[:dim]
    clip1 = clip[np.triu_indices(n_dim_obs,1)].ravel()
    clip1[picks] = 1
    clip[np.triu_indices(n_dim_obs,1)[::-1]] = clip[np.triu_indices(n_dim_obs,1)] = clip1
    
   
    thetas = np.array([np.eye(n_dim_obs) for i in range(T)])
    
    x = np.linspace(-np.pi, np.pi, T)
    plt.plot(x, np.sin((x+phase[1,2])/T**2)*(0.5/T))
    plt.show()
    for i in range(T):
        for r in range(thetas[i].shape[0]):
            for c in range(thetas[i].shape[1]):
                if r==c:
                    continue
                if clip[r,c]:
                    thetas[i,r,c] = np.sin((x[i]+phase[r,c])/T**2)*(0.5/T)
                else:
                    thetas[i,r,c] = np.sin((x[i]+phase[r,c]))*(0.5/T)
        thetas[i][clip==1] = np.clip(thetas[i][clip==1],0,1)
        thetas[i][np.abs(thetas[i])<1e-2]=0
        
        assert(is_pos_def(thetas[i]))
        theta_observed = thetas[i] - L
        assert(is_pos_def(theta_observed))
        thetas_obs = [theta_observed]

    return thetas, thetas_obs, L

In [ ]:
K, K_obs, L = generate_dataset_sin_cos(n_dim_obs=100, n_dim_lat=2,sparsity=0.99)

In [ ]:
reload(plot)
for i, theta in enumerate(K):
   # theta[np.abs(theta)<1e-2] = 0
    plot.plot_graph_with_latent_variables(theta, 0, 100, "Network at time "+str(i+1))

In [ ]:
from regain.admm import time_graph_lasso_; reload(time_graph_lasso_)
# from regain import latent_time_graph_lasso_admm; reload(latent_time_graph_lasso_admm)
Theta, S = time_graph_lasso_.time_graph_lasso(data_list, lamda=5, verbose=1, psi='l1')
# Theta, L, S = latent_time_graph_lasso_admm.time_latent_graph_lasso(data_list, alpha=5, verbose=1)

In [ ]:
from sklearn.utils.extmath import squared_norm
np.sqrt(np.sum(squared_norm(t2 - t1[2:, 2:]) for t1, t2 in zip(thetas, Theta)))

In [ ]:
from regain.forward_backward import time_graph_lasso_ as tglf; reload(tglf)
from regain import utils; reload(utils)
from regain import prox; reload(prox)
TT, SS = tglf.time_graph_lasso(data_list, lamda=5, verbose=1, max_iter=4)

In [ ]:
for i, (theta_true, theta_inferred) in enumerate(zip(thetas, Theta)):
    print(theta_true[2:, 2:])
    print(theta_inferred)
    plot_graph_with_latent_variables(theta_true[2:, 2:], 2,3, "Network at time "+str(i+1)+ " true")
    plot_graph_with_latent_variables(theta_inferred, 2,3, "Network at time "+str(i+1)+ " inferred")

In [ ]:
import sys
sys.path.insert(0, "/home/fede/src/TVGL/")

import TVGL; reload(TVGL)
thetaSet, empCovSet, status = TVGL.tvgl_hallac(data_list, lamb=5, beta=1, indexOfPenalty=3,
                                               verbose=1, max_iter=250)

In [ ]:
from sklearn.utils.extmath import squared_norm
np.sqrt(np.sum(squared_norm(t2 - t1) for t1, t2 in zip(thetas, thetaSet)))